In [ ]:
# Imports
import keras
from keras.models import Model
from keras import backend as K
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose, ZeroPadding2D
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Reshape, Dense, Flatten
import vtk
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os

# MLflow imports
import mlflow
import mlflow.keras
from mlflow.models.signature import infer_signature

# Set MLflow tracking URI
mlflow.set_tracking_uri('http://your-mlflow-server:5000')
mlflow.set_experiment('steady-flow-prediction')

In [ ]:
# Training params
batch_size = 32
epochs = 5
learning_rate = 1e-4

In [ ]:
# VTK_data class implementation remains the same
class VTK_data:
    # ... [Previous VTK_data implementation] ...
    pass

In [ ]:
# Load and prepare data
base_directory = "../data"
dataset = VTK_data(base_directory)
geometries, steady_flows = dataset.load_data()

if len(dataset.geometries) > 0 and len(dataset.steady_flows) > 0:
    train_geometries = dataset.geometries[0:dataset.split_line]
    train_steady_flows = dataset.steady_flows[0:dataset.split_line]
    test_geometries = dataset.geometries[dataset.split_line:-1]
    test_steady_flows = dataset.steady_flows[dataset.split_line:-1]
    
    train_geometries = np.stack(train_geometries, axis=0)
    train_steady_flows = np.stack(train_steady_flows, axis=0)
    test_geometries = np.stack(test_geometries, axis=0)
    test_steady_flows = np.stack(test_steady_flows, axis=0)

In [ ]:
# Model definition with MLflow tracking
with mlflow.start_run(run_name='steady-flow-unet') as run:
    # Log parameters
    mlflow.log_params({
        'batch_size': batch_size,
        'epochs': epochs,
        'learning_rate': learning_rate,
        'architecture': 'UNet',
        'input_shape': train_geometries.shape[1:],
        'output_shape': train_steady_flows.shape[1:]
    })
    
    # Model architecture
    inputs = Input(shape=(9812,))
    reshaped = Reshape((44, 223, 1))(inputs)
    
    # ... [Previous model architecture remains the same] ...
    
    model = Model(inputs=inputs, outputs=final_output)
    model.compile(
        loss='mse',
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        metrics=['MSE']
    )
    
    # Custom MLflow callback for metric logging
    class MLflowCallback(keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            logs = logs or {}
            for metric_name, metric_value in logs.items():
                mlflow.log_metric(f'epoch_{metric_name}', metric_value, step=epoch)
    
    # Train model with MLflow tracking
    history = model.fit(
        train_geometries,
        train_steady_flows,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1,
        validation_data=(test_geometries, test_steady_flows),
        callbacks=[MLflowCallback()]
    )
    
    # Evaluate model
    test_loss = model.evaluate(test_geometries, test_steady_flows, verbose=0)
    mlflow.log_metric('test_mse', test_loss[0])
    
    # Log model architecture
    mlflow.log_artifact('model_architecture.txt')
    with open('model_architecture.txt', 'w') as f:
        model.summary(print_fn=lambda x: f.write(x + '\n'))
    
    # Infer model signature
    signature = infer_signature(
        train_geometries[0:1],
        model.predict(train_geometries[0:1])
    )
    
    # Register model in registry
    mlflow.keras.log_model(
        model,
        'model',
        registered_model_name='steady_flow_predictor',
        signature=signature
    )

In [ ]:
# Visualization with prediction tracking
with mlflow.start_run(run_name='prediction-visualization'):
    predicted_steady_flow = model.predict(test_geometries, batch_size=batch_size)
    
    for i in range(min(3, predicted_steady_flow.shape[0])):
        pred_reshaped = predicted_steady_flow[i].reshape(44, 223, 2)
        true_reshaped = test_steady_flows[i].reshape(44, 223, 2)
        geom_reshaped = test_geometries[i].reshape(44, 223)
        
        plt.figure(figsize=(15, 5))
        velocity_image = np.concatenate([
            pred_reshaped[:,:,0],
            true_reshaped[:,:,0],
            geom_reshaped/10.0
        ], axis=1)
        
        plt.imshow(velocity_image)
        plt.title(f'Sample {i+1}: Predicted vs True Flow vs Geometry')
        plt.colorbar()
        
        # Log plots to MLflow
        plt.savefig(f'prediction_{i}.png')
        mlflow.log_artifact(f'prediction_{i}.png')
        plt.close()
        
        # Log prediction metrics
        mse = np.mean((pred_reshaped - true_reshaped) ** 2)
        mlflow.log_metric(f'prediction_{i}_mse', mse)